# Process

- The system launchs
- Updater runs
    - Each one is passed through detection, stored in DB
    - Show few tweets from the day
- Dashboard shows
- Streaming runs

## What happens when a tweet is found is analyzed in this notebook

What has to be done:
- store the tweets id
- clean text?
- get the model to detect it and classify it if its true
  - set the place (not included still)
- insert into twtincidents if its an incident with classification (and place)
- send information to frontend and load the tweet in the dashboard (info and map)

In [1]:
import Updater
import pandas as pd
import Detection
import Preprocessing

In [2]:
# First add everything to twtincident table
from DBConnect import DB_Connection
db = DB_Connection()
db.connect()
twt = db.query_all()
len(twt)

181485

In [3]:
twt[0]

(1391853785857859593,
 'TraficoCPanama',
 'Tráfico Panamá',
 'Se registra accidente de tránsito en la Vía Transístmica, próximo a La Cabima precaución en esta ruta. https://t.co/m6xUDtyb6g',
 datetime.datetime(2021, 5, 10, 20, 32, 56, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-300, name=None)),
 'https://www.twitter.com/TraficoCPanama/status/1391853785857859593')

In [27]:
twt_df = pd.DataFrame(twt, columns=['tweetid','user_id','user_name','text','created_at','link'])

In [28]:
# Get all classification
twt_df = Detection.get_classifications(twt_df,'text') # [twt_df.id>1383930855978979331]

In [4]:
# class incident to add to DB
class Incident:
    def __init__(self, tweetid, place, isAccident, isObstacle, isDanger):
        self.tweetid = tweetid
        self.place = place
        self.isAccident = isAccident
        self.isObstacle = isObstacle
        self.isDanger = isDanger

In [30]:
## iterate through each incident
for index, row in twt_df[twt_df.isIncident==1].iterrows():
    # create object with respective attributes
    i = Incident(row.tweetid, None, 
        True if row.isAccident==1 else False, 
        True if row.isObstacle==1 else False, 
        True if row.isDanger==1 else False)
    # insert to DB (on conflict do nothing)
    db.insert_incident(i)
    # print('Inserted - ', index)

## Updater tweepy

In [1]:
import time
from datetime import timedelta

In [2]:
# First add everything to twtincident table
from DBConnect import DB_Connection
db = DB_Connection()
db.connect()

True

In [3]:
# get last date and format it
last_date = db.query_date()
last_date = (last_date - timedelta(days=1)).strftime('%Y-%m-%d')

In [6]:
# get last week tweets
start = time.time()
past_tweets = Updater.get_tweets(from_date=last_date)
end = time.time()
print(end - start, len(past_tweets))

2.2506520748138428 54


In [7]:
## get tweets to dataframe
# dict gets a dictionary of attributes and values
data = pd.DataFrame([i.__dict__ for i in past_tweets])
## get classification and category of each tweet
data = Detection.get_classifications(data, 'text')

In [8]:
## gets max id (lastest tweet)
# this is to only process new tweets
last_id = db.query('SELECT max(inc_tweet_id) FROM public.twtincident;')[0][0]
last_id

1391853785857859593

In [9]:
## iterates through new tweets
for index, row in data[data.tweetid>last_id].iterrows():
    # inserts tweet to db
    db.insert_tweet(row)
    print('Inserted - ', index)
    # if the tweet is accident, inserts to database
    if row.isIncident==1:
        i = Incident(row.tweetid, None, True if row.isAccident==1 else False, True if row.isObstacle==1 else False, True if row.isDanger==1 else False)
        db.insert_incident(i)
        print('Incident')

## Streaming

What has to be done:
- store the tweets id
- clean text?
- get the model to detect it and classify it if its true
  - set the place (not included still)
- insert into twtincidents if its an incident with classification (and place)
- send information to frontend and load the tweet in the dashboard (info and map)